In [6]:
import json

import openai
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import json
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [7]:
def get_completion(prompt, model="gpt-4"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.5, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [8]:
# text = f"""
# The mouseleave event
# """
# prompt = f"""
# As an expert in grammar correction, find all the grammatical errors and misspellings in the following text and give the corrected sentence. If the user corpus is found to be incomplete or otherwise incorrect, return the status as False, else True. The user's corpus is delimited by triple dashes.
#
# User corpus:
# ---
# {text}
# ---
#
# Use the following output format and use JSON format with the following keys:
# Misspellings: <a python list of user's misspellings >
# Grammatical errors: <a python list of user's grammatical errors>
# the corrected sentence: <the correct sentences after fixing>
# status: <boolean value means whether the modification is successful or not>
#
# Do not return other information.
#
# """
# response = get_completion(prompt)
# print(response)

The provided user corpus does not contain any grammatical errors or misspellings. Therefore, the correction and status are as follows:

{
"Misspellings": [],
"Grammatical errors": [],
"the corrected sentence": "The mouseleave event",
"status": true
}


In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [14]:
llm = ChatOpenAI(model_name='gpt-4', temperature=0.9, request_timeout=60)
text = '{text}'
prompt = ChatPromptTemplate.from_template(text)
chain = LLMChain(llm = llm, prompt = prompt)

In [16]:
import json
import os.path as osp
from typing import Union


class Prompter_GPT(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, verbose: bool = False):
        super().__init__()

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
    ) -> str:        
        if input:
            prompt = instruction + input
        else:
            prompt = instruction
        prompt_template = prompt
        
        return prompt_template

    def get_response(self, output: str) -> str:
        return output

prompter = Prompter_GPT()

def evaluate(
    instruction,
):
    output =chain.run(prompter.generate_prompt(instruction))

    return instruction, prompter.get_response(output)

In [17]:
instruction = 'hello'
instruction, response = evaluate(instruction)
print(response)

Hello! How can I assist you today?


In [20]:
dataset_name = 'mind_small_dev'

instruction_path = f'./results/LLaMA2/{dataset_name}/results.csv'
instruction_df = pd.read_csv(instruction_path)
instruction_df

def get_instruction_ChatGPT(idx):
    instruction = instruction_df.iloc[idx]['instruction']
    return instruction

In [ ]:
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

result_save_path = f'./results/GPT3.5/{dataset_name}/'
if not os.path.exists(result_save_path):
    os.makedirs(result_save_path)

df_result = pd.DataFrame(columns=['instruction', 'response'])
# i = 0
with open(result_save_path + 'results.csv', 'w', encoding='UTF-8') as f:
        for idx in tqdm(range(instruction_df.shape[0])):
            instruction, response = evaluate(get_instruction_ChatGPT(idx))
            df_result = df_result.append({'instruction': instruction, 'response': response}, ignore_index=True)
            # print(instruction)
            # print(response)
            # break
        df_result.to_csv(f, index=False)
f.close()

  6%|▌         | 6/100 [00:16<04:08,  2.65s/it]

In [18]:
for dataset_name in ['mind_small_dev', 'steam', 'ml-100k']:
    instruction_path = f'./results/LLaMA2/{dataset_name}/results.csv'
    instruction_df = pd.read_csv(instruction_path)
    instruction_df

    def get_instruction_ChatGPT(idx):
        instruction = instruction_df.iloc[idx]['instruction']
        return instruction

    from tqdm import tqdm
    import random
    import warnings
    warnings.filterwarnings("ignore")

    result_save_path = f'./results/GPT3.5/{dataset_name}/'
    if not os.path.exists(result_save_path):
        os.makedirs(result_save_path)

    df_result = pd.DataFrame(columns=['instruction', 'response'])
    # i = 0
    with open(result_save_path + 'results.csv', 'w', encoding='UTF-8') as f:
            for idx in tqdm(range(instruction_df.shape[0])):
                instruction, response = evaluate(get_instruction_ChatGPT(idx))
                df_result = df_result.append({'instruction': instruction, 'response': response}, ignore_index=True)
                # print(instruction)
                # print(response)
                # break
            df_result.to_csv(f, index=False)
    f.close()

100%|██████████| 100/100 [14:24<00:00,  8.64s/it]
